Nama: Syarani Afa Natira Kusumah<br>
NIM: 2502037864

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import Counter
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

a. Lakukan pengelompokan dengan menggunakan pendekatan clustering, silahkan ujicoba beberapa nilai cluster (k) yang mungkin anda peroleh dari data tersebut, berikan perbandingan silhoutte score sebagai justifikasi jumlah cluster (k) yang akan anda pilih.

In [ ]:
# Load data
data = pd.read_csv("data_2D.csv")
data.head()

,Unnamed: 0,index,URL,Date,Tweet,ID,Username,Replies,Reetweets,Likes,Quotes,conversationId,Language,Links,Media,Retweeted Tweet,Bookmarks
0,0,1412,https://twitter.com/oksihen/status/16584282698...,2023-05-16 11:04:46+00:00,@__nsch @bankbsi_id @Bittersweutz kak mendinga...,1658428269849608192,oksihen,1,0,0,0,1655777758180708355,in,NaN,NaN,NaN,0
1,1,23337,https://twitter.com/reixyynx/status/1655911656...,2023-05-09 12:24:38+00:00,@Adelyaaav @med_faqih08 @bankbsi_id sore ini p...,1655911656407580672,reixyynx,0,0,0,0,1655781242166710273,in,NaN,NaN,NaN,0
2,2,23099,https://twitter.com/sheyappe/status/1655924302...,2023-05-09 13:14:54+00:00,@SeokjinRie @bankbsi_id dih ternyata transfer ...,1655924302917804033,sheyappe,1,0,0,0,1655909697399177217,in,NaN,NaN,NaN,0
3,3,5691,https://twitter.com/tfouive/status/16575732480...,2023-05-14 02:27:13+00:00,@bankbsi_id kalo ambil duitnya langsung ke ban...,1657573248035586048,tfouive,1,0,0,0,1657346196548239360,in,NaN,NaN,NaN,0
4,4,16823,https://twitter.com/itshaeyoz/status/165648391...,2023-05-11 02:18:35+00:00,@bankbsi_id min ayo dong kasi tau info ke nasa...,1656483914373005312,itshaeyoz,1,0,2,0,1655777758180708355,in,NaN,NaN,NaN,0


In [ ]:
# Removes columns other than the target
colomn_drop = list(range(0, 4)) + list(range(5, 17))

df_tweet = data.drop(data.columns[colomn_drop], axis=1)
df_tweet

,Tweet
0,@__nsch @bankbsi_id @Bittersweutz kak mendinga...
1,@Adelyaaav @med_faqih08 @bankbsi_id sore ini p...
2,@SeokjinRie @bankbsi_id dih ternyata transfer ...
3,@bankbsi_id kalo ambil duitnya langsung ke ban...
4,@bankbsi_id min ayo dong kasi tau info ke nasa...
...,...
6096,"@ratnamuninggarr @bankbsi_id ka, setauku juga ..."
6097,@bankbsi_id @fadlysuardhana 3x24 jam keterlalu...
6098,@OposisiCerdas Mau indonesia seperti bank BSI
6099,@bankbsi_id @ngebacot_bae Ga bisa di akses woo...


In [ ]:
# Function to preprocess text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'www\S+|http\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions and hashtags
    text = re.sub(r'\@\w+|\#', '', text)
    # Remove non-alphabetical characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Define Indonesian stopwords
    indonesian_stopwords = set(stopwords.words('indonesian'))
    # Tokenize and filter out stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in indonesian_stopwords]
    # Join the words back into a single string
    text = ' '.join(filtered_words)
    return text

# Apply text preprocessing to the 'Tweet' column
df_tweet['Tweet'] = df_tweet['Tweet'].apply(clean_text)
df_tweet.head()

,Tweet
0,kak mendingan hubungi aja call centernya
1,sore
2,dih transfer bank
3,kalo ambil duitnya langsung bank ga si kasian ...
4,min ayo kasi tau info nasabahnya udh sebenerny...


In [ ]:
# df_tweet.to_csv('data_preprocessing.csv')

In [ ]:
# Vectorize the tweets
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tweet_matrix = tfidf_vectorizer.fit_transform(df_tweet['Tweet'])

# Determine the optimal number of clusters using silhouette scores
sil_scores = []
cluster_range = range(2, 4)
for num_clusters in cluster_range:
    kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans_model.fit(tweet_matrix)
    cluster_labels = kmeans_model.labels_
    sil_scores.append(silhouette_score(tweet_matrix, cluster_labels))

# Find the best number of clusters
best_cluster_num = cluster_range[sil_scores.index(max(sil_scores))]
print(f'Optimal Number of Clusters: {best_cluster_num}')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Optimal Number of Clusters: 3


In [ ]:
# Cluster the data using the optimal number of clusters
final_kmeans = KMeans(n_clusters=best_cluster_num, random_state=42)
df_tweet['Cluster'] = final_kmeans.fit_predict(tweet_matrix)
df_tweet.head()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,Tweet,Cluster
0,kak mendingan hubungi aja call centernya,1
1,sore,1
2,dih transfer bank,0
3,kalo ambil duitnya langsung bank ga si kasian ...,1
4,min ayo kasi tau info nasabahnya udh sebenerny...,1


b. Lakukan persona analysis untuk cluster yang anda hasilkan, jelaskan informasi yang anda peroleh mengenai cluster tersebut berdasarkan personanya

In [ ]:
# Analysis
top_terms_per_cluster = kmeans_model.cluster_centers_.argsort()[:, ::-1]
feature_names = tfidf_vectorizer.get_feature_names_out()

df_tweet['Cluster'] = cluster_labels

# Display Tweets in each optimal cluster
for cluster_id in range(best_cluster_num):
    print(f"\nTweets in Cluster {cluster_id}:\n")
    print(df_tweet[df_tweet['Cluster'] == cluster_id]['Tweet'].head(10))


Tweets in Cluster 0:

2                                     dih transfer bank
7     tdk bpk nasabah bank bsi fans arsenal ato yg d...
9     maunya cakupan bank syariah yg diperluas sampe...
16    merketplace aja ga percaya si bsi halaaaahh ba...
21                   bsi normalllll gajianku ketundaaaa
23    alhamduliah layanan atm bsi temeninfo resmi te...
24    aceh hukum qonun bank konvensional gak beroper...
26    simpan bank dibackup bank reputasinya kantor c...
27            rek rek bank pake bsi rencana pindah bank
30           bank nasional kelas koperasi simpan pinjam
Name: Tweet, dtype: object

Tweets in Cluster 1:

0              kak mendingan hubungi aja call centernya
1                                                  sore
3     kalo ambil duitnya langsung bank ga si kasian ...
4     min ayo kasi tau info nasabahnya udh sebenerny...
5     tanda tangannya ga diatas materai tandanya rag...
6                                  kau sahabat bri cing
8                              

In [ ]:
# Display frequently occurring words in each cluster
for cluster_id in range(best_cluster_num):
    cluster_tweets = df_tweet[df_tweet['Cluster'] == cluster_id]
    all_words_in_cluster = ' '.join(cluster_tweets['Tweet']).split()
    most_common_words = Counter(all_words_in_cluster).most_common(10)
    print(f"\nMost common words in Cluster {cluster_id}:\n")
    print(most_common_words)


Most common words in Cluster 0:

[('bsi', 1435), ('bank', 1351), ('syariah', 380), ('yg', 231), ('nasabah', 170), ('indonesia', 167), ('aceh', 138), ('ya', 137), ('ga', 129), ('aja', 126)]

Most common words in Cluster 1:

[('bsi', 704), ('ya', 587), ('yg', 560), ('aja', 528), ('ga', 512), ('udah', 488), ('kak', 487), ('atm', 424), ('nya', 399), ('gak', 328)]

Most common words in Cluster 2:

[('min', 518), ('bsi', 121), ('ya', 96), ('ga', 68), ('cek', 67), ('saldo', 67), ('mobile', 59), ('transfer', 55), ('udah', 55), ('gak', 51)]


**Cluster 0**<br>
Kata-kata yang sering muncul:

- bsi: 1435
- bank: 1351
- syariah: 380
- yg: 231
- nasabah: 170
- indonesia: 167
- aceh: 138
- ya: 137
- ga: 129
- aja: 126

Analisis Persona:

Tampaknya cluster ini banyak berbicara tentang "Bank Syariah Indonesia (BSI)" dan masalah yang berkaitan dengan bank syariah. Ada banyak diskusi tentang pelanggan tentang bank ini, mungkin karena layanan atau pengalaman mereka dengannya. Kata "aceh" menunjukkan bahwa banyak percakapan mungkin berasal dari atau berkaitan dengan Aceh, wilayah yang terkenal dengan perbankan syariahnya. Kata-kata seperti "ya", "ga", dan "aja" menunjukkan bahasa yang santai dan mungkin informal.<br><br>

**Cluster 1**<br>
Kata-kata yang sering muncul:

- bsi: 704
- ya: 587
- yg: 560
- aja: 528
- ga: 512
- udah: 488
- kak: 487
- atm: 424
- nya: 399
- gak: 328

Analisis Persona:

Fokus utama tetap pada "BSI", seperti yang terlihat di cluster 0. Namun, kata seperti "kak", "ya", "ga", "aja", dan "udah" menunjukkan gaya bahasa yang sangat informal dan ramah, mungkin menunjukkan komunikasi antar pelanggan atau pelanggan dengan layanan pelanggan.Masalah ATM mungkin menjadi masalah utama di kelompok ini.<br><br>

**Cluster 2**<br>
Kata-kata yang sering muncul:

- min: 518
- bsi: 121
- ya: 96
- ga: 68
- cek: 67
- saldo: 67
- mobile: 59
- transfer: 55
- udah: 55
- gak: 51

Analisis Persona:

Cluster ini tampaknya berkonsentrasi pada interaksi dengan "min" (admin), yang sering terjadi di layanan pelanggan atau media sosial. Kata-kata seperti "cek", "saldo", "mobile", dan "transfer" menunjukkan banyak diskusi tentang layanan transaksi dan perbankan digital. Kata-kata seperti "ya", "ga", dan "udah" menunjukkan gaya bahasa informal, dan penggunaan "min" menunjukkan banyak komunikasi langsung dengan manajemen atau layanan pelanggan. Permintaan cek saldo, serta masalah dengan mobile banking dan transfer, mungkin menjadi masalah besar.


c. Anda tertarik untuk membandingkan hasil clustering dengan metode lain yaitu topic extraction, sehingga anda perlu melakukan pemodelan topic extraction. Jelaskan metode topic extraction yang anda lakukan

In [ ]:
# LDA
LDA = LatentDirichletAllocation(n_components = 3, random_state = 42)
LDA.fit(tweet_matrix)

LatentDirichletAllocation(n_components=3, random_state=42)

In [ ]:
# Function to display topics from LDA model
def show_topics(lda_model, feature_names, top_words_count):
    for idx, topic in enumerate(lda_model.components_):
        print(f"Topic {idx + 1}:")
        top_features = [feature_names[i] for i in topic.argsort()[:-top_words_count - 1:-1]]
        print(" ".join(top_features))
        print("\n")

vocab = tfidf_vectorizer.get_feature_names_out()
show_topics(LDA, vocab, 20)

Topic 1:
bank bsi yg syariah nasabah data indonesia pake pindah sistem layanan aceh kena aman bca gangguan ransomware uang orang it


Topic 2:
aja ka ga atm nya ya udh banget coba bener bsi kalo dah sih pulih nih udah kak gini ku


Topic 3:
min bsi kak saldo udah ya transfer gak mobile cek gimana error atm tarik transaksi gabisa ga tf masuk up




d. Lakukan persona analysis kembali untuk topik yang anda peroleh, bandingkan apa yang anda peroleh dari pendekatan clustering dan topic extraction. Paparkan hal-hal menarik yang berhasil anda ekstrak.

Topik 1: Berisikan keluhan gangguan layanan, migrasi sistem, dan keamanan data dan sistem.<br><br>
Topic 2: Berisikan pengalaman pribadi dengan layanan BSI, serta penggunaan ATM dan masalah yang terkait.<br><br>
Topic 3: Berisikan keluhan masalah dengan mobile banking, transfer, dan cek saldo. Pengguna sering berkomunikasi dengan layanan pelanggan untuk mendapatkan bantuan.



**Kesimpulan**:<br>
Kedua pendekatan menunjukkan bahwa isu utama adalah layanan BSI, dengan banyak diskusi tentang sistem perbankan syariah, keamanan, dan layanan digital. Namun, Clustering memberikan frekuensi kata yang lebih spesifik untuk setiap cluster, sedangkan LDA memberikan gambaran umum topik berdasarkan komponen-komponen utama.